In [1]:
import pandas as pd
import numpy as np
hplp=pd.read_csv('C:/Users/91868/Desktop/Dataset_Poverty_Prediction.csv')
hplp.shape

#need to delete the columns that were square of the other columns
#need to delete the non impacting columns in the given dataset i.e Id,idhogar
hplp.drop(columns=['Id','idhogar','SQBescolari','SQBage','SQBhogar_total',
                   'SQBedjefe','SQBhogar_nin','SQBovercrowding',
                   'SQBdependency','SQBmeaned','agesq']
                    ,inplace=True)

hplp.info()
hplp.isnull().sum()

hplp.dtypes

#Now, let's take a look at the target. We want to see the distribution of target values in the Dataset.
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 5))
plt.hist(hplp['Target'],bins=4)
plt.title('Histogram Target counts')
plt.xlabel('Target')
plt.ylabel('Values')
plt.show()

#to check the unique Values in Target Column
hplp['Target'].unique()
#We saw that, there are only 4 numerical values in for the target. Value 4 seems to dominate

#to fing the No.of Occurances of each value of Target 
pd.value_counts(hplp.Target)

#filling nulll values before test, train splitting
column=['v2a1','v18q1','rez_esc','meaneduc']
for column in hplp[column]:
    hplp[column].fillna(hplp[column].median(),inplace=True)
    
#Splitting the data into Train, Test Sets
from sklearn.model_selection import train_test_split

y=hplp['Target']                  #assigning the output variable,i.e SalePrice to Y.
hplp.drop(columns='Target',inplace=True)  #Dropping the particulat Output column "SalePrice"
X_train,X_test,y_train,y_test=train_test_split(hplp,y,test_size=0.3,random_state=50)

#need to de the Labelencode the Categorical columns
from sklearn.preprocessing import LabelEncoder

catcols=[ele for ele in X_train.columns if X_train[ele].dtype==object]

for col in catcols:
    le=LabelEncoder()
    #le.fit(X_train[col])
    X_train[col]=le.fit_transform(X_train[col])
    X_test[col]=le.transform(X_test[col])

#Implementing the Gradientboost Classifier
from sklearn.ensemble import GradientBoostingClassifier

gbc=GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.2, loss='deviance', max_depth=6,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=5, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)
gbc.fit(X_train,y_train)

print("train_Score=",gbc.score(X_train,y_train))
print("test_Score=",gbc.score(X_test,y_test))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9557 entries, 0 to 9556
Columns: 132 entries, v2a1 to Target
dtypes: float64(5), int64(124), object(3)
memory usage: 9.6+ MB


<Figure size 1200x500 with 1 Axes>

D:\ANACONDA - PYTHON\lib\site-packages\ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\ANACONDA - PYTHON\lib\site-packages\ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


train_Score= 0.9973090148004186
test_Score= 0.9170153417015342


In [4]:
#Hyperparameter tuning for the above Gradient Boosting Classifier
#to Supress the Warnings
#Randomized search CV

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

grid1={'learning_rate':[0.1,0.2,0.3,0.4],'max_depth':[3,4,5,6],'min_samples_leaf':[4,5,3,2,6]}
grid=RandomizedSearchCV(estimator=gbc,param_distributions=grid1,n_iter=5,cv=5)
grid.fit(X_train,y_train)
grid.get_params
grid.cv_results_
print(grid.best_estimator_)
print(grid.best_score_)

grid.best_params_

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.2, loss='deviance', max_depth=6,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=5, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)
0.8896696068171625


{'min_samples_leaf': 5, 'max_depth': 6, 'learning_rate': 0.2}

In [2]:
#Evaluation Metrics of the above Multiclass Classification 
from sklearn.metrics import classification_report
from sklearn.metrics import multilabel_confusion_matrix
print(multilabel_confusion_matrix(y_test,gbc.predict(X_test)))

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,gbc.predict(X_test)))

print(classification_report(y_test,gbc.predict(X_test)))

[[[2653   10]
  [  52  153]]

 [[2335   62]
  [ 108  363]]

 [[2470   31]
  [ 132  235]]

 [[ 823  220]
  [  31 1794]]]
[[ 153   21    5   26]
 [   5  363   16   87]
 [   4   21  235  107]
 [   1   20   10 1794]]
              precision    recall  f1-score   support

           1       0.94      0.75      0.83       205
           2       0.85      0.77      0.81       471
           3       0.88      0.64      0.74       367
           4       0.89      0.98      0.93      1825

    accuracy                           0.89      2868
   macro avg       0.89      0.79      0.83      2868
weighted avg       0.89      0.89      0.88      2868



In [3]:
#AUC Score
y_pred_proba=gbc.predict_proba(X_test)

from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
print(lb.fit(y_test))
print(lb.classes_)
y_test_new=lb.transform(y_test)

from sklearn.metrics import roc_auc_score
auc=roc_auc_score(y_test_new,y_pred_proba)
auc

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)
[1 2 3 4]


0.9693781396660857